In [1]:
def connection():
    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = "Riverplate121."
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
def weather_request():
    import pandas as pd
    from datetime import datetime, timedelta
    import requests
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    import re
    import pandas as pd
    from datetime import datetime
    from pytz import timezone
    
    information_df = pd.read_sql("information", con = connection())
    cities_df = pd.read_sql("cities",con= connection())
    information_df = information_df.merge(cities_df,
                                          on = "city_id",
                                          how = "left"
                                         )
    
    berlin_timezone = timezone('Europe/Berlin')
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    weather_items = []
                            
    for city in information_df["city_name"]:
        latitude = information_df.loc[information_df["city_name"] == city, "latitude"].values[0]
        longitude = information_df.loc[information_df["city_name"] == city, "longitude"].values[0]

        # Construct the API request URL
        url_map = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid=4bcaab846859609d453e249b3b3f0a16"
    
        # Make the API request and handle potential errors
        try:
            city_id_openweather = requests.get(url_map)
            city_id_openweather.raise_for_status()  # Raise an error for bad responses (non-2xx status codes)
            city_id_openweather = city_id_openweather.json()
            city_id_openweather = city_id_openweather["id"]
        
        except requests.exceptions.RequestException as e:
            # Handle request exceptions (e.g., network issues, API errors)
            print(f"Error in request for latitude {latitude}, longitude {longitude}: {e}")
            temperature_request = None  # Append None for temperature to indicate an error
            sky_request = None  # Append None for sky description to indicate an error
    
        url = f"http://api.openweathermap.org/data/2.5/forecast?id={city_id_openweather}&appid=4bcaab846859609d453e249b3b3f0a16"
 
        response = requests.get(url)
    
        response = response.json()
    
        for item in response["list"]:
            weather_item = {
                "city_name" : city,
                "forecast_time": item.get("dt_txt", None),
                "temperature": item["main"].get("temp", None),
                "forecast": item["weather"][0].get("main", None),
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                "wind_speed": item["wind"].get("speed", None),
                "retrieval_time" : retrieval_time
            }
        
            weather_items.append(weather_item)
            
    weather_df = pd.DataFrame(weather_items)
    
    weather_df = weather_df.merge(cities_df,
                                  on = "city_name",
                                  how = "left")
    
    weather_df = weather_df.drop("city_name",
                                 axis = 1
                                )
    weather_df.to_sql('weather',
                      if_exists='append',
                      con=connection(),
                      index=False
                     )
    print (weather_df)

In [8]:
def airport_schedule():
    from datetime import datetime, timedelta
    import requests
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    import re
    import pandas as pd
    from datetime import datetime
    from pytz import timezone
    
    airport_arrivals = []
    iata_names = []  # Corrected variable name
    airlines = []
    status = []
    
    airports_final = pd.read_sql("airports", con=connection())
    
    today = datetime.now().date().strftime("%Y-%m-%dT")
    tomorrow = (datetime.now().date() + timedelta(days=1)).strftime("%Y-%m-%dT")
    berlin_timezone = timezone('Europe/Berlin')
    
    current_time_berlin = datetime.now(berlin_timezone)
    time_in_12_hours = current_time_berlin + timedelta(hours=12)
    current_time_berlin_formated = current_time_berlin.strftime("%H:%M")
    time_in_12_hours_formated = time_in_12_hours.strftime("%H:%M")
        
    for airport in airports_final["Airport_IATA"]:
        
        if current_time_berlin.hour < 12:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{airport}/{today}{current_time_berlin_formated}/{today}{time_in_12_hours_formated}"
        else:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{airport}/{today}{current_time_berlin_formated}/{tomorrow}{time_in_12_hours_formated}"

        querystring = {"withLeg": "true", "withCancelled": "true", "withCodeshared": "true", "withCargo": "true",
               "withPrivate": "true", "withLocation": "false"}

        headers = {
            "X-RapidAPI-Key": "4b54815ac4mshe11ca88893efc1ep170ae5jsn49ac4b2d04a5",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            arrivals = response.json()["arrivals"]

            for arrival in arrivals:
                if "arrival" in arrival and "scheduledTime" in arrival["arrival"] and "local" in arrival["arrival"]["scheduledTime"]:
                    airport_arrivals.append(arrival["arrival"]["scheduledTime"]["local"])
                    iata_names.append(airport)  # Using the correct variable name airport
           
                    if "airline" in arrival and "name" in arrival["airline"]:
                        airlines.append(arrival["airline"]["name"])
                    else:
                        airlines.append(None)
                
                    if "status" in arrival:
                        status.append(arrival["status"])
                    else:
                        status.append(None)
        else:
            # Append None values for all arrays if the response status is not 200
            airport_arrivals.append(None)
            iata_names.append(None)
            airlines.append(None)
            status.append(None)
        
    airport_schedule_df = pd.DataFrame({"airport_arrivals": airport_arrivals, "airlines": airlines, "iata_name": iata_names, "status": status})
    
    airport_schedule_df = airport_schedule_df.merge(airports_final,
                              left_on = "iata_name",
                              right_on = "Airport_IATA",
                              how = "left"
                             )
    
    airport_schedule_df = airport_schedule_df.drop(["iata_name", "Airport_IATA"], axis=1).rename(columns={"status": "status_"})
    
    airport_schedule_df['airport_arrivals'] = pd.to_datetime(airport_schedule_df['airport_arrivals'], utc=True)

    # Format the datetime column as a string in the desired format
    airport_schedule_df['airport_arrivals'] = airport_schedule_df['airport_arrivals'].dt.strftime(
        '%Y-%m-%d %H:%M:%S')

    airport_schedule_df.to_sql('airport_sheduldes',
                                  if_exists='append',
                                  con=connection(),
                                  index=False,
                                  )
    print(airport_schedule_df)

In [9]:
weather_request()
airport_schedule()

            forecast_time  temperature forecast  rain_in_last_3h  wind_speed  \
0     2024-01-17 09:00:00       269.77    Clear              0.0        3.76   
1     2024-01-17 12:00:00       270.85   Clouds              0.0        3.50   
2     2024-01-17 15:00:00       271.71   Clouds              0.0        2.37   
3     2024-01-17 18:00:00       272.43     Snow              0.0        1.77   
4     2024-01-17 21:00:00       272.72     Snow              0.0        1.00   
...                   ...          ...      ...              ...         ...   
2035  2024-01-21 18:00:00       268.44   Clouds              0.0        2.30   
2036  2024-01-21 21:00:00       268.27   Clouds              0.0        2.13   
2037  2024-01-22 00:00:00       269.21   Clouds              0.0        2.88   
2038  2024-01-22 03:00:00       273.04   Clouds              0.0        4.03   
2039  2024-01-22 06:00:00       276.38     Rain              0.1        7.88   

           retrieval_time  city_id  cou